# Basic Ontology Interface Demo

this demonstrates the use of the BasicOntologyInterface which provides simplified access to local or remote
ontologies.

We demonstrate the use of different backends, but in practice you will likely only use one depending on your use case.

- pronto or sqlite for working with ontologies which you have a local copy of, and can trade startup time for generally faster operations
- ubergraph or ontobee or ols or bioportal for an ontology that has been loaded into a remote server

## Loading from obo files using Pronto

In [1]:
from obolib.implementations.pronto.pronto_basic_impl import ProntoImplementation
from obolib.resource import OntologyResource

### Local files

First we demonstrate loading from a file on the filesystem

In [2]:
resource = OntologyResource(slug='go-nucleus.obo', directory='input', local=True)
oi = ProntoImplementation.create(resource)

In [3]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0005773 ! vacuole
    GO:0043231 ! intracellular membrane-bounded organelle
  part_of ! part_of
    GO:0005737 ! cytoplasm


### Remote (downloading from OBO)

Next we use pronto's load from obo library feature

In [4]:
oi = ProntoImplementation.create(OntologyResource(local=False, slug='go.obo'))

In [7]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0005773 ! vacuole
    GO:0043231 ! intracellular membrane-bounded organelle
  part_of ! part of
    GO:0005737 ! cytoplasm


## SQL Database access

We can load from a SQL Database following Semantic SQL patterns

In [6]:
from obolib.implementations.sqldb.sqldb_basic_impl import SqlDatabaseBasicImpl


In [16]:
oi = SqlDatabaseBasicImpl.create(OntologyResource(slug=f'sqlite:///input/go-nucleus.db'))

CREATING: sqlite:///input/go-nucleus.db


In [20]:
## TODO
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for k, v in rels.items():
    print(f'{k} = {v}')

NotImplementedError: 

In [21]:
for curie in oi.basic_search('intracellular'):
    print(f' MATCH: {curie} ! {oi.get_label_by_curie(curie)} ')

%intracellular%
PREDS = ['rdfs:label', 'oio:hasRelatedSynonym', 'oio:hasNarrowSynonym', 'oio:hasExactSynonym', 'oio:hasBroadSynonym']
 MATCH: GO:0005622 ! intracellular anatomical structure 
 MATCH: GO:0043231 ! intracellular membrane-bounded organelle 
 MATCH: GO:0031090 ! organelle membrane 
 MATCH: GO:0043229 ! intracellular organelle 


In [22]:
oi

SqlDatabaseBasicImpl(engine=Engine(sqlite:///input/go-nucleus.db), strict=False, _session=<sqlalchemy.orm.session.Session object at 0x7f9123019f70>, _connection=None)

In [30]:
# TODO: interfaces_implemented
oic = type(oi)

In [31]:
oic.__bases__


(obolib.interfaces.basic_ontology_interface.BasicOntologyInterface,)

## Loading from OWL ontologies using owlfun

TODO

## Wrapping remote ontology portals

### OLS

TODO

### BioPortal

TODO

## Wrapping SPARQL Endpoints

### Ubergraph

TODO

### Ontobee

TODO